In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Mar  1 10:40:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')# google給的範例, 執行後會把google drive整個mount近來，路徑會多一個gdrive/MyDrive在content下
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   tiny_fddb_0301	  tiny_v3.zip	    training_example
 dataset	    tiny_fddb_0301_long   train_fddb_0301


In [3]:
# clone the Darknet
!git clone https://github.com/AlexeyAB/darknet
# 執行完後darknet資料夾會直接在content目錄下(因為現在就在content內)
%cd darknet
# 之後都在darknet資料夾裡
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make
# make darknet

Cloning into 'darknet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 14730 (delta 2), reused 7 (delta 2), pack-reused 14722
Receiving objects: 100% (14730/14730), 13.28 MiB | 24.45 MiB/s, done.
Resolving deltas: 100% (10017/10017), done.
/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 fl

In [4]:
!mkdir dataset
!unzip /mydrive/dataset/fddb_clean.zip -d dataset/

串流輸出內容已截斷至最後 5000 行。
  inflating: dataset/images/2002/09/03/big/img_14127.jpg  
  inflating: dataset/images/2002/09/03/big/img_14979.jpg  
  inflating: dataset/images/2002/09/03/big/img_14650.jpg  
  inflating: dataset/images/2002/09/03/big/img_14966.jpg  
  inflating: dataset/images/2002/09/03/big/img_14617.jpg  
  inflating: dataset/images/2002/09/03/big/img_14432.jpg  
  inflating: dataset/images/2002/09/03/big/img_14538.jpg  
  inflating: dataset/images/2002/09/03/big/img_14804.jpg  
  inflating: dataset/images/2002/09/03/big/img_14684.jpg  
  inflating: dataset/images/2002/09/03/big/img_14880.jpg  
  inflating: dataset/images/2002/09/03/big/img_14339.jpg  
  inflating: dataset/images/2002/09/03/big/img_14782.jpg  
  inflating: dataset/images/2002/09/03/big/img_14914.jpg  
  inflating: dataset/images/2002/09/03/big/img_14783.jpg  
  inflating: dataset/images/2002/09/03/big/img_14220.jpg  
  inflating: dataset/images/2002/09/03/big/img_14741.jpg  
  inflating: dataset/images/2002/09

In [5]:
import random
random.seed(888)

In [6]:
topdir = "dataset/images"
txt_file_path = topdir + "/FDDB-paths.txt"
images_list = []
with open(txt_file_path, 'r') as lines:
  for line in lines:
    line = line.strip("\n")
    images_list.append(topdir + "/" + line + ".jpg")
num_data = len(images_list)
num_train = int(num_data * 0.8)
num_test = num_data - num_train
print("total number of data: {}".format(num_data))
print("split to train: {}".format(num_train))
print("and test: {}".format(num_test))
random.shuffle(images_list)
train_list = images_list[:num_train]
test_list = images_list[num_train:]

#Create training.txt file
file = open("dataset/train.txt", "w") 
file.write("\n".join(train_list)) 
file.close() 

#Create test.txt file
file = open("dataset/test.txt", "w") 
file.write("\n".join(test_list)) 
file.close()
# debug
print("num of train images : {}".format(len(train_list)))
print("num of valid images : {}".format(len(test_list)))

total number of data: 2845
split to train: 2276
and test: 569
num of train images : 2276
num of valid images : 569


In [8]:
# Download pretrain weights darknet53
#!wget https://pjreddie.com/media/files/yolov3-tiny.weights
!cp cfg/yolov3-tiny.cfg cfg/tinyv3_fddb.cfg

!sed -i 's/batch=1/batch=64/' cfg/tinyv3_fddb.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/tinyv3_fddb.cfg
!sed -i 's/max_batches = 500200/max_batches = 20000/' cfg/tinyv3_fddb.cfg
!sed -i '22 s@steps=400000,450000@steps=16000,18000@' cfg/tinyv3_fddb.cfg
!sed -i '135 s@classes=80@classes=1@' cfg/tinyv3_fddb.cfg
!sed -i '177 s@classes=80@classes=1@' cfg/tinyv3_fddb.cfg
!sed -i '127 s@filters=255@filters=18@' cfg/tinyv3_fddb.cfg
!sed -i '171 s@filters=255@filters=18@' cfg/tinyv3_fddb.cfg
# number of anchors = 3 in each output feature map
# number of classes = 1
# filters = (4+1+1)*3 = 18
!echo -e "face" > dataset/tinyv3_fddb.names
!echo -e 'classes= 1\ntrain  = dataset/train.txt\nvalid  = dataset/test.txt\nnames = dataset/tinyv3_fddb.names\nbackup = /mydrive/tiny_fddb_0301_long/backup' > dataset/tinyv3_fddb.data

In [9]:
# cp training config
!cp dataset/tinyv3_fddb.data /mydrive/tiny_fddb_0301_long/tinyv3_fddb.data
!cp dataset/tinyv3_fddb.names /mydrive/tiny_fddb_0301_long/tinyv3_fddb.names
!cp cfg/tinyv3_fddb.cfg /mydrive/tiny_fddb_0301_long/tinyv3_fddb.cfg
!cp dataset/train.txt /mydrive/tiny_fddb_0301_long/train.txt
!cp dataset/test.txt /mydrive/tiny_fddb_0301_long/test.txt

In [11]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2021-03-01 10:50:13--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  30.2MB/s    in 1.1s    

2021-03-01 10:50:15 (30.2 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [12]:
# get weight
!./darknet partial cfg/yolov3-tiny.cfg yolov3-tiny.weights yolov3-tiny.conv.15 15

 CUDA-version: 10010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 GPU isn't used 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 Create CUDA-stream - 0 
max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3/ 1     26 x  26 x 128 ->   26 x  26 x 256 0.399 BF
   9 max 

In [13]:
!./darknet detector train dataset/tinyv3_fddb.data cfg/tinyv3_fddb.cfg yolov3-tiny.conv.15 -dont_show

串流輸出內容已截斷至最後 5000 行。

 19903: 0.373210, 0.231616 avg loss, 0.000010 rate, 0.562356 seconds, 1273792 images, 0.039617 hours left
Loaded: 0.000041 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.840897), count: 9, class_loss = 0.146496, iou_loss = 0.176787, total_loss = 0.323283 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.011959, iou_loss = 0.000000, total_loss = 0.011959 
 total_bbox = 2195855, rewritten_bbox = 0.024819 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.925153), count: 4, class_loss = 0.000008, iou_loss = 0.025492, total_loss = 0.025500 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000003, iou_loss = 0.000000, total_loss = 0.000003 
 total_bbox = 2195859, rewritten_bbox = 0.024819 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 A

In [ ]:
!pwd

/content/darknet


In [ ]:
test_list[0]

'dataset/images/2002/08/01/big/img_2152.jpg'

In [ ]:
!echo -e "face" > dataset/tinyv3_fddb.names

In [ ]:
!./darknet detector test dataset/tinyv3_fddb.data cfg/tinyv3_fddb.cfg /mydrive/tiny_fddb_0301/backup/tinyv3_fddb_last.weights dataset/images/2002/08/01/big/img_2152.jpg

 CUDA-version: 10010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 co

In [ ]:
test_list[5]

'dataset/images/2002/08/10/big/img_191.jpg'

In [ ]:
!./darknet detector test dataset/tinyv3_fddb.data cfg/tinyv3_fddb.cfg /mydrive/tiny_fddb_0301/backup/tinyv3_fddb_last.weights dataset/images/2002/08/10/big/img_191.jpg

 CUDA-version: 10010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 co